To start, here's several helpful packages to load

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns           # statistical data visualization

In [ ]:
#User ratings
ratings_file = 'datasets/movies_rs/ratings.csv'
df_ratings = pd.read_csv(ratings_file)

#Movies
movies_file = 'datasets/movies_rs/movies.csv'
df_movies = pd.read_csv(movies_file)

We need a dataset that contains the userId, movie title and its ratings.

In [ ]:
movie_data = pd.merge(df_ratings, df_movies, on='movieId')

Now we group the dataset by the title of the movie and then calculate the mean of the rating for each movie


In [ ]:
movie_data.groupby('title')['rating'].mean().sort_values(ascending=False).head()

Let's sort the ratings in the descending order of their average ratings

In [ ]:
movie_data.groupby('title')['rating'].count().sort_values(ascending=False).head()

 A movie can make it to the top of the above list even if only a single user has given it five stars. Therefore, the above stats can be misleading. Normally, a movie which is really a good one gets a higher rating by a large number of users.

In [ ]:
movie_data.groupby('title')['rating'].count().sort_values(ascending=False).head()

We will create a new dataframe called ratings_mean_count and first add the average rating of each movie to this dataframe and we will add the number of ratings for a movie to the ratings_mean_count dataframe as follows

In [ ]:
ratings_mean_count = pd.DataFrame(movie_data.groupby('title')['rating'].mean())
ratings_mean_count['rating_counts'] = pd.DataFrame(movie_data.groupby('title')['rating'].count())


To understand better the data, let's plot a histogram for the number of ratings represented by the rating_counts column in the above dataframe.

In [ ]:
plt.figure(figsize=(10,8))
plt.rcParams['patch.force_edgecolor'] = True
ratings_mean_count['rating_counts'].hist(bins=50)

From the above plot, we can see that most of the movies have received less than 50 ratings and there are no movies having more than 100 ratings.
What about the average ratings?

In [ ]:
plt.figure(figsize=(10,8))
plt.rcParams['patch.force_edgecolor'] = True
ratings_mean_count['rating'].hist(bins=50)

The data shows a weak normal distribution with the mean of around 3.5. There are a few outliers in the data as well.
Movies with a higher number of ratings usually have a high average rating as well since a good movie is normally well-known and a well-known movie is watched by a large number of people, and thus usually has a higher rating.
Let's see if this is also the case with the movies in our dataset. We will plot average ratings against the number of ratings.

In [ ]:
plt.figure(figsize=(10,8))
plt.rcParams['patch.force_edgecolor'] = True
sns.jointplot(x='rating', y='rating_counts', data=ratings_mean_count, alpha=0.4)

The graph shows that, in general, movies with higher average ratings actually have more number of ratings, compared with movies that have lower average ratings.

FINDING SIMILARITIES

We will use the correlation between the ratings of a movie as the similarity metric.

To find the correlation between the ratings of the movie, we need to create a matrix where each column is a movie name and each row contains the rating assigned by a specific user to that movie.

This matrix will have a lot of null values since every movie is not rated by every user.

In [ ]:
user_movie_rating = movie_data.pivot_table(index='userId', columns='title', values='rating')


We know that each column contains all the user ratings for a particular movie.
Now, let's find all the user ratings for the movie Forrest Gump (1994) and find the movies similar to it.
We chose this movie since it has the highest number of ratings and we want to find the correlation between movies that have a higher number of ratings.

In [ ]:
movie_name = 'Forrest Gump (1994)'
movie_X_ratings = user_movie_rating[movie_name]

We can find the correlation between the user ratings for the Forest Gump (1994) and all the other movies using corrwith() function.

In [ ]:
movies_like_movie_X = user_movie_rating.corrwith(movie_X_ratings, method='pearson')

corr_movie_X = pd.DataFrame(movies_like_movie_X, columns=['Correlation'])
corr_movie_X.dropna(inplace=True)


Now, let's sort the movies in descending order of correlation to see highly correlated movies at the top.

In [ ]:
corr_movie_X.sort_values('Correlation', ascending=False)

From the above output, we can see that the movies that have high correlation with Forrest Gump (1994) are not very well known.
This shows that correlation alone is not a good metric for similarity because there can be a user who watched 'Forest Gump (1994) and only one other movie and rated both of them as 5.
A solution to this problem is to retrieve only those correlated movies that have at least more than 50 ratings.

In [ ]:
corr_movie_X = corr_movie_X.join(ratings_mean_count['rating_counts'])

Now, let's now filter movies correlated to Forest Gump (1994), that have more than 50 ratings.

In [ ]:
corr_movie_X[corr_movie_X ['rating_counts']>50].sort_values('Correlation', ascending=False).head(10)